# Voronoi Tesselations 

This is an attempt at computing power diagrams in Python (spoilers: it doesn't work so well). Unlike in R, there is no ready to use package to compute additivey weighted Voronoi cells. If you manage to write a code, I'd be happy to see it! 

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

import numpy as np
import random as rd

from scipy.spatial import Voronoi, voronoi_plot_2d, ConvexHull, HalfspaceIntersection

from shapely import geometry, ops, wkt

There is no option in Python to crop Voronoi cells between $[0,1]^2$. The trick to obtain edges that do not go to infinity is to add 4 very distant points.

In [ ]:
rd.seed(777)
nCells = 10

y1 = np.random.uniform(0,1,nCells)
y2 = np.random.uniform(0,1,nCells)

print(y1)
print(y2)

centroids = list(zip(y1,y2))
centroids.append((100,100))
centroids.append((-100,100))
centroids.append((100,-100))
centroids.append((-100,-100))

In [ ]:
vor = Voronoi(centroids)
voronoi_plot_2d(vor, show_vertices=True)
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.show()

A function to compute Voronoi cells' area:

In [ ]:
def voronoi_areas(v):
    area = np.zeros(v.npoints)
    polygon = []
    square = geometry.Polygon([(0,0),(0,1),(1,1),(1,0)])
    for i, reg_num in enumerate(v.point_region):
        indices = v.regions[reg_num]
        polygon = geometry.MultiPoint(v.vertices[indices]).convex_hull
        trunc_polygon = square.intersection(polygon)
        area[i] = trunc_polygon.area
        if abs(v.points[i][0]) == 100:
            area[i]=0
    return area

In [ ]:
demand = voronoi_areas(vor)

My attemps at computing power diagrams: I follow [this paper](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.104.678&rep=rep1&type=pdf) which itself is based on [Aurenhammer's paper](https://www.cs.jhu.edu/~misha/Spring16/Aurenhammer87.pdf) and try to computed 2D weighted Voronoi tesselations as the intersection of 3D unweighted Voronoi tesselations with a plane.

In [ ]:
class vor_3dto2d:
    npoints = int
    points = [[]]
    vertices = [[]]
    ridge_points = [[]]
    ridge_vertices = [[]]
    regions = [[]]
    point_region = []
    
    def __init__(self,vor): 
        self.npoints = vor.npoints
        self.points = vor.points[:,0:2]
        self.vertices = vor.vertices[:,0:2]
        self.ridge_points = vor.ridge_points
        self.ridge_vertices = vor.ridge_vertices
        self.regions = vor.regions
        self.point_region = vor.point_region

In [ ]:
vtilde = np.repeat(0.00,nCells)
weights = np.sqrt(np.max(vtilde)-vtilde)
centroids_3d = list(zip(y1,y2,weights))
centroids_3d.append((100,100,100))
centroids_3d.append((-100,100,100))
centroids_3d.append((100,-100,100))
centroids_3d.append((-100,-100,100))
centroids_3d.append((100,100,-100))
centroids_3d.append((-100,100,-100))
centroids_3d.append((100,-100,-100))
centroids_3d.append((-100,-100,-100))

In [ ]:
vor_3d_noweights = Voronoi(centroids_3d)
vor_2d_noweights = vor_3dto2d(vor_3d_noweights)

voronoi_plot_2d(vor_2d_noweights, show_vertices=True)
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.show()

In [ ]:
MAX_ITER = 2
PREC = 1e-2
CONT = True
niter = 0

In [ ]:
q = np.repeat(1/nCells,nCells)
vtilde = np.repeat(0,nCells)
epsilon = 0.1

In [ ]:
while (CONT == True and niter < MAX_ITER):

    weights = np.sqrt(np.max(vtilde)-vtilde)
    print(weights)
    centroids_3d = list(zip(y1,y2,weights))
    centroids_3d.append((100,100,100))
    centroids_3d.append((-100,100,100))
    centroids_3d.append((100,-100,100))
    centroids_3d.append((-100,-100,100))
    centroids_3d.append((100,100,-100))
    centroids_3d.append((-100,100,-100))
    centroids_3d.append((100,-100,-100))
    centroids_3d.append((-100,-100,-100))

    vor_3d = Voronoi(centroids_3d)
    vor_2d = vor_3dto2d(vor_3d)

    voronoi_plot_2d(vor_2d, show_vertices=True)
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.show()
    
    demand = voronoi_areas(vor_2d)[0:10]

    if (max(abs(demand-q))<PREC/nCells):
        CONT = False
    else:
        vtilde = vtilde - epsilon * (demand - q)

    niter = niter +1
